# Simple Readout Fidelity Measurement 

Run an identity, `I`, or an `X` pulse to estimate readout fidelity. 

## setup

In [ ]:
from math import pi
from typing import List

import numpy as np

from pyquil import Program, get_qc
from pyquil.api import QuantumComputer
from pyquil.gates import *

## measuring readout fidelity

`run_readout` constructs and measures a program for a single qubit depending on whether the $\left|0\right>$ state or the $\left|1\right>$ state is targeted. When $\left|0\right>$ is targeted, we prepare a qubit in the ground state with an identity operation. Similarly, When $\left|1\right>$ is targeted, we prepare a qubit in the excited state with an `X` gate. 

In [ ]:
def run_readout(qc: QuantumComputer, qubit: int, target: int, n_shots: int = 1000):
    """
    Measure a qubit. Optionally flip a bit first.

    :param qc: The quantum computer to run on
    :param qubit: The qubit to probe
    :param target: What state we want. Either 0 or 1.
    :param n_shots: The number of times to repeat the experiment
    """

    # Step 2. Construct and compile your program
    program = Program()
    ro = program.declare('ro', 'BIT', 1)
    # Uncomment to enable active reset
    # program += RESET()
    if target == 0:
        program += I(qubit)
    elif target == 1:
        program += RX(pi, qubit)
    else:
        raise ValueError("Target should be 0 or 1")

    program += MEASURE(qubit, ro[0])
    program = program.wrap_in_numshots_loop(n_shots)

    nq_program = program
    # Uncomment to test the quilc compiler
    # nq_program = qc.compiler.quil_to_native_quil(program)
    executable = qc.compiler.native_quil_to_executable(nq_program)

    bitstrings = qc.run(executable)
    return np.mean(bitstrings[:, 0])

`run_readouts` is a wrapper for `run_readout` that runs on each qubit, targetting $\left|0\right>$ for many measurements (default 1000) and then $\left|1\right>$. When we prepare in the $\left|0\right>$ state, we expect to measure in the $\left|0\right>$ state, so the percentage of the time we do in fact measure $\left|0\right>$ gives us `p(0|0)`. We do the same for `p(1|1)` and print the results. 

In [ ]:
def run_readouts(qc: QuantumComputer):
    """
    Characterize readout on several qubits, one at a time.

    This collects results and prints them out at the end because we
    print the compiled and uncompiled programs within ``run_readout``.

    :param qc: The QuantumComputer to run on
    :param qubits: A list of qubits to characterize one at a time.
    """
    results = []
    for qubit in qc.qubits():
        p0 = run_readout(qc=qc, qubit=qubit, target=0)
        p1 = run_readout(qc=qc, qubit=qubit, target=1)
        results += [(qubit, p0, p1)]

    print('q     p(0|0)    p(1|1)')
    for qubit, p0, p1 in results:
        print(f'q{qubit:<3d}{1-p0:10f}{p1:10f}')

In [ ]:
run_readouts(qc=get_qc('9q-square-noisy-qvm'))